In [1]:
from dotenv import load_dotenv

load_dotenv()

True

In [4]:
import os
print(f'''Env Variables
LANGCHAIN_TRACING_V2: {os.environ["LANGCHAIN_TRACING_V2"]}
LANGCHAIN_PROJECT: {os.environ["LANGCHAIN_PROJECT"]}
LANGCHAIN_ENDPOINT: {os.environ["LANGCHAIN_ENDPOINT"]}
''')

Env Variables
LANGCHAIN_TRACING_V2: true
LANGCHAIN_PROJECT: machine-learning-workshop
LANGCHAIN_ENDPOINT: https://api.smith.langchain.com



## LangSmithのトレースの基本
traceableデコレータを使用することで、任意の関数の引数と返り値をLangSmithで確認できるようになる。

In [20]:
from IPython.display import display, Markdown
from langsmith import traceable
import openai

openai_client = openai.Client()

@traceable
def format_prompt(question):
    return [
        {
            "role": "system",
            "content": "あなたはBigQueryのエキスパートです. 出したいデータのクエリを作成してください. 出力はクエリのみで他の情報は不要です.",
        },
        {
            "role": "user",
            "content": f"{question}"
        }
    ]

@traceable(run_type="llm")
def invoke_llm(messages):
    return openai_client.chat.completions.create(
        messages=messages, model="gpt-4o", temperature=0
    )

@traceable
def parse_output(response):
    return response.choices[0].message.content

@traceable
def run_pipeline():
    messages = format_prompt("ウェブサイトの回遊率")
    response = invoke_llm(messages)
    return parse_output(response)

display(Markdown(run_pipeline()))

```sql
SELECT
  user_id,
  COUNT(DISTINCT page_id) AS pages_visited,
  COUNT(DISTINCT session_id) AS sessions,
  COUNT(DISTINCT page_id) / COUNT(DISTINCT session_id) AS page_views_per_session
FROM
  `your_dataset.your_table`
GROUP BY
  user_id
```

openaiとのやり取りの可観測にするラッパー `wrap_openai` を使うと詳細な情報を簡単に取得可能になる。

In [32]:
from langsmith.wrappers import wrap_openai

wrap_openai_client = wrap_openai(openai.Client())

@traceable(name="run_pipeline with wrap_openai")
def run_pipeline_with_wrap_llm():
    messages = format_prompt("ウェブサイトの回遊率")
    response = wrap_openai_client.chat.completions.create(
        messages=messages, model="gpt-4o", temperature=0
    )
    return parse_output(response)

display(Markdown(run_pipeline_with_wrap_llm()))

```sql
SELECT
  user_id,
  COUNT(DISTINCT session_id) AS total_sessions,
  COUNT(DISTINCT CASE WHEN page_viewed THEN session_id END) AS sessions_with_page_views,
  (COUNT(DISTINCT CASE WHEN page_viewed THEN session_id END) / COUNT(DISTINCT session_id)) AS bounce_rate
FROM
  `your_dataset.your_table`
GROUP BY
  user_id
```

LCEL (LangChain Expression Language) を使えば、LangSmithでの観測が楽にできる。

In [58]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI

model = ChatOpenAI(model="gpt-4")

prompt = ChatPromptTemplate.from_messages([
    ("system", "あなたはBigQueryのエキスパートです. 出したいデータのクエリを作成してください. 出力はクエリのみで他の情報は不要です."),
    ("human", "{question}")
])
output_parser = StrOutputParser()

chain = prompt | model | output_parser
display(Markdown(chain.invoke("ウェブサイトの回遊率")))

```
# まずは全体のセッション数と、1ページ以上訪れたセッション数を取得
WITH total_sessions AS (
  SELECT COUNT(*) as total_sessions
  FROM `project.dataset.sessions`
),
pages_per_session AS (
  SELECT sessionId, COUNT(*) as num_pages
  FROM `project.dataset.page_views`
  GROUP BY sessionId
),
sessions_with_multiple_pages AS (
  SELECT COUNT(*) as sessions_with_multiple_pages
  FROM pages_per_session
  WHERE num_pages > 1
)
# 回遊率を計算（1ページ以上訪れたセッション数 / 全体のセッション数）
SELECT sessions_with_multiple_pages / total_sessions as bounce_rate
FROM total_sessions, sessions_with_multiple_pages
```

このクエリは、ウェブサイトの回遊率を計算するためのものです。回遊率とは、ユーザーがウェブサイトの複数のページを訪れたセッションの割合を指します。

まず、全体のセッション数を`total_sessions`サブクエリで取得します。次に、各セッションで訪れたページ数を`pages_per_session`サブクエリで取得します。そして、1ページ以上訪れたセッション数を`sessions_with_multiple_pages`サブクエリで取得します。

最後に、1ページ以上訪れたセッション数を全体のセッション数で割ることで、回遊率を計算します。

なお、このクエリはBigQueryのStandard SQL構文を使用しています。また、このクエリはサンプルであり、実際のデータスキーマによっては修正が必要です。

In [61]:
import textwrap

from langsmith import Client
from langsmith.schemas import Run, Example
from langsmith.evaluation import evaluate

ls_client = Client() # LangSmithのクライアント

# 作成するデータセット
dataset_name = "SQL Samples"

# データセットがあれば削除
if ls_client.has_dataset(dataset_name=dataset_name):
    dataset = ls_client.delete_dataset(dataset_name=dataset_name)

dataset = ls_client.create_dataset(dataset_name, description="ML Workshop用のサンプルクエリ")

# データセットにexampleを保存
ls_client.create_examples(
    inputs=[
        {"question": "MAUを取得"},
        {"question": "新規ユーザ数の推移"},
    ],
    outputs=[
        {"query": textwrap.dedent("""
           SELECT
               COUNT(DISTINCT user_id) AS monthly_active_users
           FROM
               `your_dataset.your_table`
           WHERE
               activity_date BETWEEN DATE_SUB(CURRENT_DATE(), INTERVAL 1 MONTH) AND CURRENT_DATE()
        """)},
        {"query": textwrap.dedent("""
            SELECT
                signup_date,
                COUNT(user_id) AS new_users
            FROM
                `your_dataset.your_table`
            GROUP BY
                signup_date
            ORDER BY
            　　 signup_date
        """)},
    ],
    dataset_id=dataset.id,
)

In [64]:
dataset

Dataset(name='SQL Samples', description='ML Workshop用のサンプルクエリ', data_type=<DataType.kv: 'kv'>, id=UUID('24833e3e-e5ca-4a21-a8bf-57ba81c38d83'), created_at=datetime.datetime(2024, 6, 22, 6, 38, 11, 940342, tzinfo=datetime.timezone.utc), modified_at=datetime.datetime(2024, 6, 22, 6, 38, 11, 940342, tzinfo=datetime.timezone.utc), example_count=0, session_count=0, last_session_start_time=None)

example_countが0となっているので、LangSmith Clientを使ってdatasetを読み直す

In [62]:
ls_client.read_dataset(dataset_name=dataset_name)

Dataset(name='SQL Samples', description='ML Workshop用のサンプルクエリ', data_type=<DataType.kv: 'kv'>, id=UUID('24833e3e-e5ca-4a21-a8bf-57ba81c38d83'), created_at=datetime.datetime(2024, 6, 22, 6, 38, 11, 940342, tzinfo=datetime.timezone.utc), modified_at=datetime.datetime(2024, 6, 22, 6, 38, 11, 940342, tzinfo=datetime.timezone.utc), example_count=2, session_count=0, last_session_start_time=None)

In [65]:
dataset.url

'https://smith.langchain.com/o/bd14a154-65e7-52b4-bdce-b9a16d5e3513/datasets/24833e3e-e5ca-4a21-a8bf-57ba81c38d83'

In [66]:
ls_client.create_examples(
    inputs=[
        {"question": "月ごとのCV数の推移"},
    ],
    outputs=[
        {"query": textwrap.dedent("""
           SELECT
               FORMAT_TIMESTAMP('%Y-%m', conv_date) AS conversion_month,
               COUNT(conv_id) AS conversions
           FROM
               `your_dataset.your_table`
           WHERE
               conv_date BETWEEN DATE_SUB(CURRENT_DATE(), INTERVAL 1 YEAR) AND CURRENT_DATE()
           GROUP BY
               conversion_month
           ORDER BY
               conversion_month
        """)},
    ],
    dataset_id=dataset.id,
)

exampleを増やしたことにより、datasetのバージョンも変更されている

In [71]:
ls_client.read_dataset(dataset_name=dataset_name).example_count

3

In [73]:
# datasetに保存されているexampleの一覧
list(ls_client.list_examples(dataset_name=dataset_name))

[Example(dataset_id=UUID('24833e3e-e5ca-4a21-a8bf-57ba81c38d83'), inputs={'question': '月ごとのCV数の推移'}, outputs={'query': "\nSELECT\n    FORMAT_TIMESTAMP('%Y-%m', conv_date) AS conversion_month,\n    COUNT(conv_id) AS conversions\nFROM\n    `your_dataset.your_table`\nWHERE\n    conv_date BETWEEN DATE_SUB(CURRENT_DATE(), INTERVAL 1 YEAR) AND CURRENT_DATE()\nGROUP BY\n    conversion_month\nORDER BY\n    conversion_month\n"}, metadata=None, id=UUID('46c9945b-28a6-416e-9390-7f3f47d83c3d'), created_at=datetime.datetime(2024, 6, 22, 6, 56, 46, 321500, tzinfo=datetime.timezone.utc), modified_at=datetime.datetime(2024, 6, 22, 6, 56, 46, 321500, tzinfo=datetime.timezone.utc), runs=[], source_run_id=None),
 Example(dataset_id=UUID('24833e3e-e5ca-4a21-a8bf-57ba81c38d83'), inputs={'question': 'MAUを取得'}, outputs={'query': '\nSELECT\n    COUNT(DISTINCT user_id) AS monthly_active_users\nFROM\n    `your_dataset.your_table`\nWHERE\n    activity_date BETWEEN DATE_SUB(CURRENT_DATE(), INTERVAL 1 MONTH) AND C

In [74]:
# # LLMが呼ばれたときに自動的にトレースするようにする
# openai_client = wrap_openai(openai.Client())

# # inputsにexampleが1つずつ渡される
# def predict(inputs: dict) -> dict:
#     messages = [{"role": "user", "content": inputs["question"]}] # datasetのinputキー (question) と合わせる
#     response = openai_client.chat.completions.create(messages=messages, model="gpt-4o")
#     return {"output": response}

# # Define evaluators
# def must_mention(run: Run, example: Example) -> dict:
#     prediction = run.outputs.get("output") or ""
#     print(f"run id: {run.id}\n")
#     required = example.outputs.get("must_mention") or [] # outputsのキー (must_mention) と合わせる
#     score = all(phrase in prediction for phrase in required) # scoreは自分で定義したものでよい
#     return {"key":"must_mention", "score": 0, "comment": "comment test"} # key, score, commentを返す

# experiment_results = evaluate(
#     predict, # Your AI system
#     data=dataset_name, # The data to predict and grade over
#     evaluators=[must_mention], # The evaluators to score the results
#     experiment_prefix="rap-generator", # A prefix for your experiment names to easily identify them
#     metadata={
#       "version": "1.0.0",
#     },
# )